In [ ]:
mamba activate chrombpnet2
export LD_LIBRARY_PATH=/cluster/project/treutlein/jjans/software/miniforge3/envs/cuda11_env/lib:$CONDA_PREFIX/lib:$LD_LIBRARY_PATH

In [ ]:
parallel -j 4 '
    modisco motifs -i celltype_models_human/modelv1_{1}/chrombpnet_nobias.profile_scores_avg.h5 -n 1000000 -o celltype_models_human/modelv1_{1}/modisco_avg/{1}.h5
    
    mkdir -p celltype_models_human/modelv1_{1}/modisco_avg/MEME
    mkdir -p celltype_models_human/modelv1_{1}/modisco_avg/JASPAR

    modisco report -i celltype_models_human/modelv1_{1}/modisco_avg/{1}.h5 -o celltype_models_human/modelv1_{1}/modisco_avg/JASPAR/ -s celltype_models_human/modelv1_{1}/modisco_avg/JASPAR/ -m JASPAR2022_CORE_vertebrates_non-redundant_pfms_meme.txt

    modisco report -i celltype_models_human/modelv1_{1}/modisco_avg/{1}.h5 -o celltype_models_human/modelv1_{1}/modisco_avg/MEME/ -s celltype_models_human/modelv1_{1}/modisco_avg/MEME/ -m motifs.meme.txt

' ::: "${cell_types[@]}" 


In [50]:
modisco report -i celltype_models_human/modelv1_enterocytes/modisco_avg/enterocytes.h5 -o celltype_models_human/modelv1_enterocytes/modisco_avg/ -s celltype_models_human/modelv1_enterocytes/modisco_avg/ -m JASPAR2022_CORE_vertebrates_non-redundant_pfms_meme.txt

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.



In [52]:
modisco report -i celltype_models_human/modelv1_enterocytes/modisco_avg/enterocytes.h5 -o celltype_models_human/modelv1_enterocytes/modisco_avg/ -s celltype_models_human/modelv1_enterocytes/modisco_avg/meme -m motifs.meme.txt

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.



In [1]:
mamba activate finemo_gpu

In [2]:
unset PYTHONPATH

In [3]:
which python

/cluster/project/treutlein/jjans/software/miniforge3/envs/finemo_gpu/bin/python


In [4]:
export PYTHONPATH=/cluster/project/treutlein/jjans/software/miniforge3/envs/finemo_gpu/lib/python3.10/site-packages:$PYTHONPATH


In [5]:
finemo extract-regions-chrombpnet-h5 -h

usage: finemo extract-regions-chrombpnet-h5 [-h] -c H5S [H5S ...] -o OUT_PATH
                                            [-w REGION_WIDTH]

options:
  -h, --help            show this help message and exit
  -c H5S [H5S ...], --h5s H5S [H5S ...]
                        One or more H5 files of contribution scores, with
                        paths delimited by whitespace. Scores are averaged
                        across files. (default: None)
  -o OUT_PATH, --out-path OUT_PATH
                        The path to the output .npz file. (default: None)
  -w REGION_WIDTH, --region-width REGION_WIDTH
                        The width of the input region centered around each
                        peak summit. (default: 1000)


In [ ]:
celltype_models_human/modelv1_enterocytes/fold_0/output/auxiliary/interpret_subsample/chrombpnet_nobias.profile_scores.h5

In [3]:
finemo extract-regions-chrombpnet-h5 -c celltype_models_human/modelv1_enterocytes/fold_0/output/auxiliary/interpret_subsample/chrombpnet_nobias.profile_scores.h5 -o test_finemo

In [ ]:
finemo extract-regions-chrombpnet-h5 -c celltype_models_human/modelv1_enterocytes/fold_0/output/auxiliary/interpret_subsample/chrombpnet_nobias.profile_scores.h5 -o test_finemo

In [ ]:
modisco motifs [-h] -i H5PY -n MAX_SEQLETS -op OUTPUT_PREFIX [-l N_LEIDEN] [-w WINDOW] [-v]

In [26]:
ls celltype_models_human/modelv1_EEC/chrombpnet_nobias.profile_scores_avg.h5

chrombpnet_nobias.profile_scores_avg.h5  fold_0  fold_1  fold_2  fold_3  fold_4


In [62]:
cell_types=("Stem_cell" "Goblet_cell" "EEC" "enterocytes")
folds=("fold_1" "fold_2" "fold_3" "fold_4" "fold_0")

# Function to create output directory if it doesn't exist
create_output_dir() {
    local output_dir="$1"
    mkdir -p $output_dir
}

# Export the function so that GNU Parallel can access it
export -f create_output_dir


parallel -j 1 '
    finemo extract-regions-chrombpnet-h5 -c celltype_models_human/modelv1_{1}/chrombpnet_nobias.profile_scores_avg.h5 -o finemo/npz_files/{1}_avg

    mkdir -p finemo/{1}_avg/{2}_patterns
    
    finemo call-hits -r finemo/npz_files/{1}_avg.npz -m celltype_models_human/modelv1_{1}/modisco_avg/{1}.h5 -p celltype_models_human/modelv1_{1}/fold_0/output/auxiliary/interpret_subsample/chrombpnet_nobias.interpreted_regions.bed -o finemo/{1}_avg/{2}_patterns
' ::: "${cell_types[@]}" ::: "${cell_types[@]}"


In [63]:
ls finemo

EEC_avg      Goblet_cell_avg  Stem_cell_avg    npz_files
Goblet_cell  Stem_cell	      enterocytes_avg


In [7]:
finemo call-hits -r finemo/npz_files/enterocytes_avg.npz -m celltype_models_human/modelv1_enterocytes/modisco_avg/enterocytes.h5 -p celltype_models_human/modelv1_enterocytes/fold_0/output/auxiliary/interpret_subsample/chrombpnet_nobias.interpreted_regions.bed -o finemo/enterocytes_avg/enterocytes_patterns_manual/


100%|██████████████████████████████| 30000/30000 [00:10<00:00, 2840.56regions/s]


In [8]:
finemo call-hits -r finemo/npz_files/enterocytes_avg.npz -m celltype_models_human/modelv1_enterocytes/fold_0/output/auxiliary/interpret_subsample/modisco_results_profile_scores.h5 -p celltype_models_human/modelv1_enterocytes/fold_0/output/auxiliary/interpret_subsample/chrombpnet_nobias.interpreted_regions.bed -o finemo/enterocytes_avg/enterocytes_patterns_manual/testing


100%|██████████████████████████████| 30000/30000 [00:19<00:00, 1571.75regions/s]


In [ ]:
finemo call-hits -r finemo/npz_files/enterocytes_avg.npz -m celltype_models_human/modelv1_enterocytes/modisco_avg/enterocytes.h5 -p celltype_models_human/modelv1_enterocytes/fold_0/output/auxiliary/interpret_subsample/chrombpnet_nobias.interpreted_regions.bed -o finemo/enterocytes_avg/enterocytes_patterns_manual/


In [ ]:
finemo call-hits -r test_finemo.npz -m celltype_models_human/modelv1_enterocytes/fold_0/output/auxiliary/interpret_subsample/modisco_results_profile_scores.h5 -p celltype_models_human/modelv1_enterocytes/fold_0/output/auxiliary/interpret_subsample/chrombpnet_nobias.interpreted_regions.bed -o test_finemo

In [24]:
cell_types=("Stem_cell" "Goblet_cell" "EEC" "enterocytes")
folds=("fold_1" "fold_2" "fold_3" "fold_4" "fold_0")

# Function to create output directory if it doesn't exist
create_output_dir() {
    local output_dir="$1"
    mkdir -p $output_dir
}

# Export the function so that GNU Parallel can access it
export -f create_output_dir


parallel -j 1 '
    finemo extract-regions-chrombpnet-h5 -c celltype_models_human/modelv1_{1}/{2}/output/auxiliary/interpret_subsample/chrombpnet_nobias.profile_scores.h5 -o finemo/npz_files/{1}_{2}

    mkdir -p finemo/{1}/{2}
    
    finemo call-hits -r finemo/npz_files/{1}_{2}.npz -m modisco_summary_{3}.hdf5 -p celltype_models_human/modelv1_{1}/{2}/output/auxiliary/interpret_subsample/chrombpnet_nobias.interpreted_regions.bed -o finemo/{1}/{2}/{3} -b 500
' ::: "${cell_types[@]}" ::: "${folds[@]}" ::: "${cell_types[@]}"


/cluster/project/treutlein/USERS/jjans/software/finemo_gpu/src/finemo/hitcaller.py:275: RuntimeWarning: Optimizer failed for region 23908.
  warnings.warn(f"Optimizer failed for region {ind}.", RuntimeWarning)
/cluster/project/treutlein/USERS/jjans/software/finemo_gpu/src/finemo/hitcaller.py:275: RuntimeWarning: Optimizer failed for region 3697.
  warnings.warn(f"Optimizer failed for region {ind}.", RuntimeWarning)
/cluster/project/treutlein/USERS/jjans/software/finemo_gpu/src/finemo/hitcaller.py:275: RuntimeWarning: Optimizer failed for region 20930.
  warnings.warn(f"Optimizer failed for region {ind}.", RuntimeWarning)
/cluster/project/treutlein/USERS/jjans/software/finemo_gpu/src/finemo/hitcaller.py:275: RuntimeWarning: Optimizer failed for region 23908.
  warnings.warn(f"Optimizer failed for region {ind}.", RuntimeWarning)
/cluster/project/treutlein/USERS/jjans/software/finemo_gpu/src/finemo/hitcaller.py:275: RuntimeWarning: Optimizer failed for region 28671.
  warnings.warn(f"Opti

In [21]:
ls modisco_summary*

modisco_summary.hdf5		  modisco_summary_Stem_cell.hdf5
modisco_summary_EEC.hdf5	  modisco_summary_enterocytes.hdf5
modisco_summary_Goblet_cell.hdf5  modisco_summary_test.hdf5


In [12]:
finemo call-hits -r test_finemo.npz -m celltype_models_human/modelv1_enterocytes/fold_0/output/auxiliary/interpret_subsample/modisco_results_profile_scores.h5 -p celltype_models_human/modelv1_enterocytes/fold_0/output/auxiliary/interpret_subsample/chrombpnet_nobias.interpreted_regions.bed -o test_finemo

100%|███████████████████████████████| 30000/30000 [01:47<00:00, 279.50regions/s]


In [13]:
finemo extract-regions-chrombpnet-h5 -c celltype_models_human/modelv1_Goblet_cell/fold_0/output/auxiliary/interpret_subsample/chrombpnet_nobias.profile_scores.h5 -o test_finemo_GC


finemo call-hits -r test_finemo_GC.npz -m celltype_models_human/modelv1_enterocytes/fold_0/output/auxiliary/interpret_subsample/modisco_results_profile_scores.h5 -p celltype_models_human/modelv1_Goblet_cell/fold_0/output/auxiliary/interpret_subsample/chrombpnet_nobias.interpreted_regions.bed -o test_finemo_GC

100%|███████████████████████████████| 30000/30000 [01:38<00:00, 305.74regions/s]
